<a href="https://colab.research.google.com/github/27ABH/AI-Data-Science-2025-26/blob/main/Poem_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading AI model...


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
def generate_strict_imagery_poem(topic):
    # We use a purely descriptive starting point
    prompt = f"Nature Scene: {topic}\nDescription: The"

    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # We find the IDs for the words we want to BAN
    # This prevents "he says", "I'm", and dialogue markers
    forbidden_words = [" he", " says", " said", "I", "I'm", '"', "'", "!", " safety"]
    bad_words_ids = [tokenizer.encode(w) for w in forbidden_words]

    print(f"--- Generating Poem: {topic} ---\n")

    output_ids = model.generate(
        input_ids,
        max_new_tokens=60,
        do_sample=True,
        temperature=0.7,
        no_repeat_ngram_size=3,
        bad_words_ids=bad_words_ids, # THIS BANS THE DIALOGUE
        repetition_penalty=2.0,
        pad_token_id=tokenizer.eos_token_id
    )

    full_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # We strip the prompt and split by words to reconstruct 4 lines
    content = full_text.replace(prompt, "").strip()
    words = content.split()

    # Since the model struggles with newlines, we'll force-wrap the text
    # into 4 lines of roughly 5-7 words each.
    line_length = 6
    for i in range(4):
        start = i * line_length
        end = start + line_length
        line_text = " ".join(words[start:end])
        if line_text:
            print(f"Line {i+1}: {line_text}")

    print("\n--- Done ---")

generate_strict_imagery_poem("falling autumn leaves")

--- Generating Poem: falling autumn leaves ---

Line 1: tree will slowly begin to fall.
Line 2: It is a beautiful little green
Line 3: apple, with the sweet yellow and
Line 4: red flowers on top of it

--- Done ---
